The following jupyter notebook contains the computation for the Nested Shapley values included in the appendix example in the paper Alonso et al. 

In [11]:
import numpy as np
from itertools import combinations
import pandas as pd
from src.NestedShapley import create_Ci_list, get_VS_matrix, create_N_M_list, get_Ci_matrix, get_NM_matrix, get_shapley_node, get_approx_shapley_node, associate_shapley_agent
from src.ClusteringAlgorithm import SilhouetteScoreRange, GenerateArray, KMeansAlgorithm
from src.appendix_model import model_p2p
import pytz
import os

In [2]:
# Directory with data
current_directory = os.path.normpath(os.path.join(os.getcwd(), ".."))
file_path_data = os.path.join(current_directory,r"data_appendix")

# Input data
start_date_str = "2019-6-11"
end_date_str = "2019-6-12"

N=12 # number of agents

np.random.seed(42)  # You can choose any number as a seed
houses_pv = np.random.choice(range(1, N + 1), int(N/2+2), replace=False)
houses_pv = [f"{i}" for i in houses_pv]

houses_bat = ['12', '10', '8', '9']

Import and preprocess the demand data

In [6]:
def import_demand(file_demand,N):
    date_format_str = '%Y-%m-%d %H:%M:%S%z'  # '2019-12-06 14:00:00+00:00' format
    demand = pd.read_csv(file_demand, index_col=0,
                         parse_dates=[0], date_format=date_format_str)

    utc_tz = pytz.UTC  # just used to ensure matching the dates with the index
    start_date = pd.to_datetime(start_date_str, format='%Y-%m-%d').tz_localize(utc_tz)
    end_date = pd.to_datetime(end_date_str, format='%Y-%m-%d').tz_localize(utc_tz)
    demand.index = demand.index.to_pydatetime()
    demand = demand[demand.columns[:N]]
    demand = demand[(demand.index >= start_date) & (demand.index < end_date)]
    
    return demand
    
file_demand = os.path.join(file_path_data,"demand_Jan_365days.csv")
demand = import_demand(file_demand,N)
demand_array, demand_ = GenerateArray(demand)
demand_.head()

,Total_Consumption,Ave_Monthly_Peak
house,,
1,9.321,1.275
10,27.996,2.053
11,18.465,2.730
12,31.581,3.215
2,7.759,0.566


Functions

In [23]:
def agents_preprocessing(dataKMeans):
    agent_list = [] #index identifies the number of the agent, value identifies the name of the agent
    # dataKMeans_sorted = dataKMeans.sort_values(by='predicted cluster', ascending=True)
    for i in dataKMeans["predicted cluster"].unique():
        df_ = dataKMeans[dataKMeans["predicted cluster"]==i]
        agent_list.extend(df_.index.to_list())

    # check how many agents in each node
    agents = dataKMeans["predicted cluster"].value_counts().to_numpy()

    return agent_list, agents

def generate_data_dict_case_1(file_path_data, start_date_str, end_date_str, demand, current_agents, houses_pv, houses_bat):
    list_houses = current_agents.copy()

    list_houses_pv = [i for i in houses_pv if i in list_houses]
    capacity_pv = list(np.full(len(list_houses_pv), 5))
    list_houses_bat = [i for i in houses_bat if i in list_houses]

    # transforming dates to align with data
    utc_tz = pytz.UTC  # just used to ensure matching the dates with the index
    start_date = pd.to_datetime(start_date_str, format='%Y-%m-%d').tz_localize(utc_tz)
    end_date = pd.to_datetime(end_date_str, format='%Y-%m-%d').tz_localize(utc_tz)

    # Get spot prices
    date_format_str = '%Y-%m-%d %H:%M:%S%z'  # '2019-12-06 14:00:00+00:00' format
    file_path = os.path.join(file_path_data, r"dayahead_Jan_365days.csv")
    P_spot_df = pd.read_csv(file_path, index_col=0,
                            parse_dates=[0], date_format=date_format_str)  # to make sure the date is read properly
    P_spot_df.index = P_spot_df.index.to_pydatetime() # convert to a datetime format required for the model
    P_spot_df = P_spot_df[["day ahead price (p/kWh)"]]  # get only price in pences/kWh
    P_spot_df_ = P_spot_df[(P_spot_df.index >= start_date) & (P_spot_df.index < end_date)]
    # Convert the dataframe P_spot_df_ to dictionary for data input for the function model_p2p()
    P_spot = P_spot_df_.to_dict()

    # Get demand
    demand_ = demand[list_houses]  # Filter based on the houses selected
    demand_ = demand_.stack()  # Set time and household as index
    # Convert the dataframe to dictionary
    P_demand = demand_.to_dict()

    # Get solar profiles, we assume the PV profile is the same for each house given that they are located close to each other
    file_path = os.path.join(file_path_data, r"solar_profile_scenarios_yearly.csv")
    PV_df = pd.read_csv(file_path, index_col=0,
                        parse_dates=[0], date_format=date_format_str)
    PV_df.index = PV_df.index.to_pydatetime() # convert to a datetime format required for the model
    scn = "1"
    PV_df = PV_df[[scn]]  # Select just one scenario, the data is prepared for several scenarios
    PV_df_ = PV_df[(PV_df.index >= start_date) & (PV_df.index < end_date)]
    # Convert the dataframe to dictionary
    PV = PV_df_.to_dict()

    # Set T
    list_T = P_spot_df_.index.to_list()

    # Parameter PV_cap
    PV_cap = {f"{key}":capacity_pv[i] for i, key in enumerate(list_houses_pv)}

    # Scalars (single value parameters)
    Psi = 1 - 0.076  # Losses (assume a loss of 7.6% through the local network, Luth)
    Mu_c = 0.96  # Charging efficiency
    Mu_d = 0.96  # Discharging efficiency
    Alpha = 1.5  # charging rate 2.5 kW -> 1.25 kWh/hour at constant rate
    Beta = 1.5  # discharging rate 2.5 kW -> 1.25 kWh/hour at constant rate
    Smax = 4  # capacity batteries [kWh] # It can also be changes to be similar to parameter PV_cap where you specify the capacity of each battery
    Smin = Smax * 0.2  # minimum state of charge of batteries at all times
    S_init = Smax * 0.5  # initial state of charge of the battery
    c_FFR = 1000

    # Construct data dictionary
    data = {  # always start with None and then dictionary
        None: {  # names of the keys equal to the name of the parameteres in the model
            'H': {None: list_houses},  # providing data for set H
            'H_pv': {None: list_houses_pv},  # providing data for set H_pv
            "H_bat": {None: list_houses_bat},  # providing data for set H_bat
            "T": {None: list_T},  # providing datetime for set T
            # Parameters
            'P_spot': P_spot['day ahead price (p/kWh)'],
            "PV": PV[scn],
            "PV_cap": PV_cap,
            "Dem": P_demand,
            # Scalars
            "Psi": {None: Psi},
            "Mu_c": {None: Mu_c},
            "Mu_d": {None: Mu_d},
            "Alpha": {None: Alpha},
            "Beta": {None: Beta},
            "Smax": {None: Smax},
            "Smin": {None: Smin},
            "S_init": {None: S_init},
            "c_FFR": {None: c_FFR}
        }}

    return data

def generate_data_dict_case_2(file_path_data, start_date_str, end_date_str, demand, current_agents):
    list_houses = current_agents.copy()

    list_houses_pv = list_houses.copy()
    capacity_pv = list(np.full(len(list_houses_pv), 5)) # Capacity of 5
    list_houses_bat = list_houses.copy()

    # transforming dates to align with data
    utc_tz = pytz.UTC  # just used to ensure matching the dates with the index
    start_date = pd.to_datetime(start_date_str, format='%Y-%m-%d').tz_localize(utc_tz)
    end_date = pd.to_datetime(end_date_str, format='%Y-%m-%d').tz_localize(utc_tz)

    # Get spot prices
    date_format_str = '%Y-%m-%d %H:%M:%S%z'  # '2019-12-06 14:00:00+00:00' format
    file_path = os.path.join(file_path_data, r"dayahead_Jan_365days.csv")
    P_spot_df = pd.read_csv(file_path, index_col=0,
                            parse_dates=[0], date_format=date_format_str)  # to make sure the date is read properly
    P_spot_df.index = P_spot_df.index.to_pydatetime() # convert to a datetime format required for the model
    P_spot_df = P_spot_df[["day ahead price (p/kWh)"]]  # get only price in pences/kWh
    P_spot_df_ = P_spot_df[(P_spot_df.index >= start_date) & (P_spot_df.index < end_date)]
    # Convert the dataframe P_spot_df_ to dictionary for data input for the function model_p2p()
    P_spot = P_spot_df_.to_dict()

    # Get demand
    demand_ = demand[list_houses]  # Filter based on the houses selected
    demand_ = demand_.stack()  # Set time and household as index
    # Convert the dataframe to dictionary
    P_demand = demand_.to_dict()

    # Get solar profiles, we assume the PV profile is the same for each house given that they are located close to each other
    file_path = os.path.join(file_path_data, r"solar_profile_scenarios_yearly.csv")
    PV_df = pd.read_csv(file_path, index_col=0,
                        parse_dates=[0], date_format=date_format_str)
    PV_df.index = PV_df.index.to_pydatetime() # convert to a datetime format required for the model
    scn = "1"
    PV_df = PV_df[[scn]]  # Select just one scenario, the data is prepared for several scenarios
    PV_df_ = PV_df[(PV_df.index >= start_date) & (PV_df.index < end_date)]
    # Convert the dataframe to dictionary
    PV = PV_df_.to_dict()

    # Set T
    list_T = P_spot_df_.index.to_list()

    # Parameter PV_cap
    PV_cap = {f"{key}":capacity_pv[i] for i, key in enumerate(list_houses_pv)}

    # Scalars (single value parameters)
    Psi = 1 - 0.076  # Losses (assume a loss of 7.6% through the local network, Luth)
    Mu_c = 0.96  # Charging efficiency
    Mu_d = 0.96  # Discharging efficiency
    Alpha = 1.5  # charging rate 2.5 kW -> 1.25 kWh/hour at constant rate
    Beta = 1.5  # discharging rate 2.5 kW -> 1.25 kWh/hour at constant rate
    Smax = 4  # capacity batteries [kWh] # It can also be changes to be similar to parameter PV_cap where you specify the capacity of each battery
    Smin = Smax * 0.2  # minimum state of charge of batteries at all times
    S_init = Smax * 0.5  # initial state of charge of the battery
    c_FFR = 1000

    # Construct data dictionary
    data = {  # always start with None and then dictionary
        None: {  # names of the keys equal to the name of the parameteres in the model
            'H': {None: list_houses},  # providing data for set H
            'H_pv': {None: list_houses_pv},  # providing data for set H_pv
            "H_bat": {None: list_houses_bat},  # providing data for set H_bat
            "T": {None: list_T},  # providing datetime for set T
            # Parameters
            'P_spot': P_spot['day ahead price (p/kWh)'],
            "PV": PV[scn],
            "PV_cap": PV_cap,
            "Dem": P_demand,
            # Scalars
            "Psi": {None: Psi},
            "Mu_c": {None: Mu_c},
            "Mu_d": {None: Mu_d},
            "Alpha": {None: Alpha},
            "Beta": {None: Beta},
            "Smax": {None: Smax},
            "Smin": {None: Smin},
            "S_init": {None: S_init},
            "c_FFR": {None: c_FFR}
        }}

    return data


## Computations for tree 2-X

In [14]:
# generate clusters according to tree
n_clusters = 2
sil_score_dict, sample_silhouette_values_dict, centers_dict = SilhouetteScoreRange(array=demand_array, nClusters = n_clusters)
dataKMeans, dataProcessed = KMeansAlgorithm(array=demand_array, dataframe=demand_, nClusters=n_clusters)

# Get agents ordered and number of agents in each node. Note that the tree only has two layers
agents_list, agents = agents_preprocessing(dataKMeans) # agents_list contains the name of the agent and the element it corresponds
demand = demand[agents_list] # reorganise the demand to match the clustering tree

# Prepare ordered tree
tree_layers = [2,6,0] # number of children nodes per node
special_layer_index = 1  # Index of the special layer (0-based). This layer does not have the number of children node indicated in tree_layers

Ci_list = create_Ci_list(tree_layers, special_layer=special_layer_index, special_children=list(agents))
Ci = get_Ci_matrix(Ci_list)

# Construct the necessary matrices for computing appropriately the Nested Shapley. See basic_shapley.py for a simple example.
N_M_list = create_N_M_list(Ci_list, Ci)
N_M = get_NM_matrix(N_M_list)

VS = get_VS_matrix(Ci)

A = np.dot(VS,N_M)

dataKMeans["predicted cluster"]

house
1     0
10    1
11    1
12    1
2     0
3     0
4     0
5     0
6     1
7     0
8     0
9     1
Name: predicted cluster, dtype: int32

### Case 1: Different agent contribution

In Case 1, we explore the case where agents do not contribute equally in terms of generation and storage technologies. Those community members owning these technologies are indicated by the variables `houses_pv` and `houses_bat`.

In [15]:
# Calculate the characteristic function for case with all agents contributing the same
# Initialise v to save the characteristic functions of each subcoalition
v = np.zeros(len(A))
for row in range(len(A)):
    print(f"Run {row} of {len(A)}")
    current_agents_index = A[row]
    current_agents = [agent for i, agent in enumerate(agents_list) if current_agents_index[i]==1]

    n_houses = np.sum(current_agents == 1)

    # Create dictionary of data with function generate_data_dict()
    data = generate_data_dict_case_1(file_path_data, start_date_str, end_date_str, demand, current_agents, houses_pv, houses_bat)

    # Run the model
    instance = model_p2p(data)
    v[row] = instance.objective_function.expr()

# Activate to save the value functions in a csv file
#np.savetxt(r"v_NestedShapley.csv", v, delimiter=",", fmt="%f")

nodal_shapley = get_shapley_node(Ci, A, VS, v)
nested_shapley_node = get_approx_shapley_node(Ci, nodal_shapley)
nested_shapley_agents = associate_shapley_agent(N_M=N_M, approx_shapley_node=nested_shapley_node)

Run 0 of 161
Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-08
Read LP format model from file C:\Users\raquelal\AppData\Local\Temp\tmpbv6xuilr.pyomo.lp
Reading time = 0.02 seconds
x5954: 3649 rows, 5857 columns, 12240 nonzeros
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 3649 rows, 5857 columns and 12240 nonzeros
Model fingerprint: 0x15af60e2
Coefficient statistics:
  Matrix range     [9e-01, 1e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-02, 4e+00]
Presolve removed 3265 rows and 3361 columns
Presolve time: 0.01s
Presolved: 384 rows, 2496 columns, 4655 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   9.521375e+01   0.000000e+00      0s
     602    1.3815442e+02   0.000000e+00   0.000000e+00      0s

Solved in 602 iterations and 0.03 se

In [20]:
print("For Case 1: Different agent contribution")
print("The Nested Shapley values of the agents are: \n")
for i, agent in enumerate(agents_list):
    print(f"Agent {agent}: {nested_shapley_agents[i].round(1)}")

For Case 1: Equal agent contribution
The Nested Shapley values of the agents are: 

Agent 1: 3.9
Agent 2: -1.5
Agent 3: 29.1
Agent 4: 55.4
Agent 5: 26.6
Agent 7: 13.1
Agent 8: 10.8
Agent 10: 32.0
Agent 11: 39.4
Agent 12: 43.8
Agent 6: 13.1
Agent 9: 7.3


### Case 2: Equal agent contribution

In this case study, we assume every house contributes equally to the coalition in storage and generation technologies. Each agent is assumed to own a PV panel of 5 kW and a storage technology of 4 kWh.

In [24]:
# Calculate the characteristic function for case with all agents contributing the same
# Initialise v to save the characteristic functions of each subcoalition
v = np.zeros(len(A))
for row in range(len(A)):
    print(f"Run {row} of {len(A)}")
    current_agents_index = A[row]
    current_agents = [agent for i, agent in enumerate(agents_list) if current_agents_index[i]==1]

    n_houses = np.sum(current_agents == 1)

    # Create dictionary of data with function generate_data_dict()
    data = generate_data_dict_case_2(file_path_data, start_date_str, end_date_str, demand, current_agents)

    # Run the model
    instance = model_p2p(data)
    v[row] = instance.objective_function.expr()

# Activate to save the value functions in a csv file
#np.savetxt(r"v_NestedShapley.csv", v, delimiter=",", fmt="%f")

nodal_shapley = get_shapley_node(Ci, A, VS, v)
nested_shapley_node = get_approx_shapley_node(Ci, nodal_shapley)
nested_shapley_agents = associate_shapley_agent(N_M=N_M, approx_shapley_node=nested_shapley_node)

Run 0 of 161
Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-08
Read LP format model from file C:\Users\raquelal\AppData\Local\Temp\tmpzgl26rmg.pyomo.lp
Reading time = 0.03 seconds
x7394: 5089 rows, 6721 columns, 15114 nonzeros
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 5089 rows, 6721 columns and 15114 nonzeros
Model fingerprint: 0xcf916613
Coefficient statistics:
  Matrix range     [9e-01, 1e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-03, 4e+00]
Presolve removed 4417 rows and 3361 columns
Presolve time: 0.01s
Presolved: 672 rows, 3360 columns, 6377 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   7.491203e+01   0.000000e+00      0s
     850    1.5484582e+01   0.000000e+00   0.000000e+00      0s

Solved in 850 iterations and 0.03 se

In [25]:
print("For Case 2: Equal agent contribution")
print("The Nested Shapley values of the agents are: \n")
for i, agent in enumerate(agents_list):
    print(f"Agent {agent}: {nested_shapley_agents[i].round(1)}")

For Case 2: Equal agent contribution
The Nested Shapley values of the agents are: 

Agent 1: -0.3
Agent 2: -0.4
Agent 3: 2.5
Agent 4: 1.1
Agent 5: -0.1
Agent 7: -0.9
Agent 8: 2.9
Agent 10: 30.0
Agent 11: 18.2
Agent 12: 40.2
Agent 6: -5.6
Agent 9: 7.9


## Computations for tree 3-X

In [26]:
# generate clusters according to tree
n_clusters = 3
sil_score_dict, sample_silhouette_values_dict, centers_dict = SilhouetteScoreRange(array=demand_array, nClusters = n_clusters)
dataKMeans, dataProcessed = KMeansAlgorithm(array=demand_array, dataframe=demand_, nClusters=n_clusters)

# Get agents ordered and number of agents in each node
agents_list, agents = agents_preprocessing(dataKMeans) # agents_list contains the name of the agent and the element it corresponds
demand = demand[agents_list] # reorganise the demand to match the clustering tree

# Prepare ordered tree
tree_layers = [3,5,0]
special_layer_index = 1  # Index of the special layer (0-based)

Ci_list = create_Ci_list(tree_layers, special_layer=special_layer_index, special_children=list(agents))
Ci = get_Ci_matrix(Ci_list)

# Construct the necessary matrices for computing appropriately the Nested Shapley. See basic_shapley.py for a simple example.
N_M_list = create_N_M_list(Ci_list, Ci)
N_M = get_NM_matrix(N_M_list)

VS = get_VS_matrix(Ci)

A = np.dot(VS,N_M)

### Case 1: Different agent contribution

In [28]:
# Calculate the characteristic function
# Initialise v to save the characteristic functions of each subcoalition
v = np.zeros(len(A))
for row in range(len(A)):
    print(f"Run {row} of {len(A)}")
    current_agents_index = A[row]
    current_agents = [agent for i, agent in enumerate(agents_list) if current_agents_index[i]==1]

    n_houses = np.sum(current_agents == 1)

    # Create dictionary of data with function generate_data_dict_case_1()
    data = generate_data_dict_case_1(file_path_data, start_date_str, end_date_str, demand, current_agents, houses_pv, houses_bat)

    # Run the model
    instance = model_p2p(data)
    v[row] = instance.objective_function.expr()

# Activate to save the value functions in a csv file
#np.savetxt(r"v_NestedShapley.csv", v, delimiter=",", fmt="%f")

nodal_shapley = get_shapley_node(Ci, A, VS, v)
nested_shapley_node = get_approx_shapley_node(Ci, nodal_shapley)
nested_shapley_agents = associate_shapley_agent(N_M=N_M, approx_shapley_node=nested_shapley_node)

Run 0 of 72
Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-08
Read LP format model from file C:\Users\raquelal\AppData\Local\Temp\tmpdp977aal.pyomo.lp
Reading time = 0.01 seconds
x3362: 2209 rows, 3265 columns, 6960 nonzeros
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2209 rows, 3265 columns and 6960 nonzeros
Model fingerprint: 0x9c4c3974
Coefficient statistics:
  Matrix range     [9e-01, 1e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-02, 4e+00]
Presolve removed 1921 rows and 1921 columns
Presolve time: 0.00s
Presolved: 288 rows, 1344 columns, 2447 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   8.317197e+01   0.000000e+00      0s
     511    1.2488064e+02   0.000000e+00   0.000000e+00      0s

Solved in 511 iterations and 0.01 secon

In [29]:
print("For Case 1: Different agent contribution")
print("The Nested Shapley values of the agents are: \n")
for i, agent in enumerate(agents_list):
    print(f"Agent {agent}: {nested_shapley_agents[i].round(1)}")

For Case 1: Different agent contribution
The Nested Shapley values of the agents are: 

Agent 1: -5.5
Agent 3: 23.2
Agent 4: 58.6
Agent 5: 28.6
Agent 8: 15.6
Agent 10: 31.6
Agent 11: 38.9
Agent 12: 43.3
Agent 6: 13.0
Agent 9: 7.2
Agent 2: 7.9
Agent 7: 10.6


### Case 2: Equal agent contribution

In [30]:
# Calculate the characteristic function for case with all agents contributing the same
# Initialise v to save the characteristic functions of each subcoalition
v = np.zeros(len(A))
for row in range(len(A)):
    print(f"Run {row} of {len(A)}")
    current_agents_index = A[row]
    current_agents = [agent for i, agent in enumerate(agents_list) if current_agents_index[i]==1]

    n_houses = np.sum(current_agents == 1)

    # Create dictionary of data with function generate_data_dict()
    data = generate_data_dict_case_2(file_path_data, start_date_str, end_date_str, demand, current_agents)

    # Run the model
    instance = model_p2p(data)
    v[row] = instance.objective_function.expr()

# Activate to save the value functions in a csv file
#np.savetxt(r"v_NestedShapley.csv", v, delimiter=",", fmt="%f")

nodal_shapley = get_shapley_node(Ci, A, VS, v)
nested_shapley_node = get_approx_shapley_node(Ci, nodal_shapley)
nested_shapley_agents = associate_shapley_agent(N_M=N_M, approx_shapley_node=nested_shapley_node)

Run 0 of 72
Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-08
Read LP format model from file C:\Users\raquelal\AppData\Local\Temp\tmpi0fxucsr.pyomo.lp
Reading time = 0.01 seconds
x4322: 3169 rows, 3841 columns, 8876 nonzeros
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 3169 rows, 3841 columns and 8876 nonzeros
Model fingerprint: 0x33537d7b
Coefficient statistics:
  Matrix range     [9e-01, 1e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-03, 4e+00]
Presolve removed 2689 rows and 1921 columns
Presolve time: 0.00s
Presolved: 480 rows, 1920 columns, 3595 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   6.271465e+01   0.000000e+00      0s
     643    2.1720477e+01   0.000000e+00   0.000000e+00      0s

Solved in 643 iterations and 0.01 secon

In [31]:
print("For Case 2: Equal agent contribution")
print("The Nested Shapley values of the agents are: \n")
for i, agent in enumerate(agents_list):
    print(f"Agent {agent}: {nested_shapley_agents[i].round(1)}")

For Case 2: Equal agent contribution
The Nested Shapley values of the agents are: 

Agent 1: -2.0
Agent 3: 10.3
Agent 4: 7.2
Agent 5: -2.6
Agent 8: 6.5
Agent 10: 29.6
Agent 11: 18.0
Agent 12: 39.6
Agent 6: -5.5
Agent 9: 7.7
Agent 2: -6.7
Agent 7: -6.7


## Computations for tree 4-X

In [ ]:
# generate clusters according to tree
n_clusters = 4
sil_score_dict, sample_silhouette_values_dict, centers_dict = SilhouetteScoreRange(array=demand_array, nClusters = n_clusters)
dataKMeans, dataProcessed = KMeansAlgorithm(array=demand_array, dataframe=demand_, nClusters=n_clusters)

# Get agents ordered and number of agents in each node
agents_list, agents = agents_preprocessing(dataKMeans) # agents_list contains the name of the agent and the element it corresponds
demand = demand[agents_list] # reorganise the demand to match the clustering tree

# Prepare ordered tree
tree_layers = [4,3,0]
special_layer_index = 1  # Index of the special layer (0-based)

Ci_list = create_Ci_list(tree_layers, special_layer=special_layer_index, special_children=list(agents))
Ci = get_Ci_matrix(Ci_list)

# Construct the necessary matrices for computing appropriately the Nested Shapley. See basic_shapley.py for a simple example.
N_M_list = create_N_M_list(Ci_list, Ci)
N_M = get_NM_matrix(N_M_list)

VS = get_VS_matrix(Ci)

A = np.dot(VS,N_M)

### Case 1: Different agent contribution

In [ ]:
# Calculate the characteristic function
# Initialise v to save the characteristic functions of each subcoalition
v = np.zeros(len(A))
for row in range(len(A)):
    print(f"Run {row} of {len(A)}")
    current_agents_index = A[row]
    current_agents = [agent for i, agent in enumerate(agents_list) if current_agents_index[i]==1]

    n_houses = np.sum(current_agents == 1)

    # Create dictionary of data with function generate_data_dict_case_1()
    data = generate_data_dict_case_1(file_path_data, start_date_str, end_date_str, demand, current_agents, houses_pv, houses_bat)

    # Run the model
    instance = model_p2p(data)
    v[row] = instance.objective_function.expr()

# Activate to save the value functions in a csv file
#np.savetxt(r"v_NestedShapley.csv", v, delimiter=",", fmt="%f")

nodal_shapley = get_shapley_node(Ci, A, VS, v)
nested_shapley_node = get_approx_shapley_node(Ci, nodal_shapley)
nested_shapley_agents = associate_shapley_agent(N_M=N_M, approx_shapley_node=nested_shapley_node)

In [ ]:
print("For Case 1: Different agent contribution")
print("The Nested Shapley values of the agents are: \n")
for i, agent in enumerate(agents_list):
    print(f"Agent {agent}: {nested_shapley_agents[i].round(1)}")

### Case 2: Equal agent contribution

In [ ]:
# Calculate the characteristic function for case with all agents contributing the same
# Initialise v to save the characteristic functions of each subcoalition
v = np.zeros(len(A))
for row in range(len(A)):
    print(f"Run {row} of {len(A)}")
    current_agents_index = A[row]
    current_agents = [agent for i, agent in enumerate(agents_list) if current_agents_index[i]==1]

    n_houses = np.sum(current_agents == 1)

    # Create dictionary of data with function generate_data_dict()
    data = generate_data_dict_case_2(file_path_data, start_date_str, end_date_str, demand, current_agents)

    # Run the model
    instance = model_p2p(data)
    v[row] = instance.objective_function.expr()

# Activate to save the value functions in a csv file
#np.savetxt(r"v_NestedShapley.csv", v, delimiter=",", fmt="%f")

nodal_shapley = get_shapley_node(Ci, A, VS, v)
nested_shapley_node = get_approx_shapley_node(Ci, nodal_shapley)
nested_shapley_agents = associate_shapley_agent(N_M=N_M, approx_shapley_node=nested_shapley_node)

In [ ]:
print("For Case 2: Equal agent contribution")
print("The Nested Shapley values of the agents are: \n")
for i, agent in enumerate(agents_list):
    print(f"Agent {agent}: {nested_shapley_agents[i].round(1)}")

## Computations for tree 6-X

In [ ]:
# generate clusters according to tree
n_clusters = 6
sil_score_dict, sample_silhouette_values_dict, centers_dict = SilhouetteScoreRange(array=demand_array, nClusters = n_clusters)
dataKMeans, dataProcessed = KMeansAlgorithm(array=demand_array, dataframe=demand_, nClusters=n_clusters)

# Get agents ordered and number of agents in each node
agents_list, agents = agents_preprocessing(dataKMeans) # agents_list contains the name of the agent and the element it corresponds
demand = demand[agents_list] # reorganise the demand to match the clustering tree

# Prepare ordered tree. This case manually created based on the results from the k-means algorithm
Ci_list = [
    [2,3,4,5,6,7],
    [8,9,10],
    [11,12,13],
    [14,15],
    [16,17],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    []
]

Ci = get_Ci_matrix(Ci_list)

N_M_list = [
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 [1, 2, 3],
 [4, 5, 6],
 [7, 8],
 [9, 10],
 [11],
 [12],
 [1],
 [2],
 [3],
 [4],
 [5],
 [6],
 [7],
 [8],
 [9],
 [10]]

# Construct the necessary matrices for computing appropriately the Nested Shapley. See basic_shapley.py for a simple example.
N_M_list = create_N_M_list(Ci_list, Ci)
N_M = get_NM_matrix(N_M_list)

VS = get_VS_matrix(Ci)

A = np.dot(VS,N_M)

### Case 1: Different agent contribution

In [ ]:
# Calculate the characteristic function
# Initialise v to save the characteristic functions of each subcoalition
v = np.zeros(len(A))
for row in range(len(A)):
    print(f"Run {row} of {len(A)}")
    current_agents_index = A[row]
    current_agents = [agent for i, agent in enumerate(agents_list) if current_agents_index[i]==1]

    n_houses = np.sum(current_agents == 1)

    # Create dictionary of data with function generate_data_dict_case_1()
    data = generate_data_dict_case_1(file_path_data, start_date_str, end_date_str, demand, current_agents, houses_pv, houses_bat)

    # Run the model
    instance = model_p2p(data)
    v[row] = instance.objective_function.expr()

# Activate to save the value functions in a csv file
#np.savetxt(r"v_NestedShapley.csv", v, delimiter=",", fmt="%f")

nodal_shapley = get_shapley_node(Ci, A, VS, v)
nested_shapley_node = get_approx_shapley_node(Ci, nodal_shapley)
nested_shapley_agents = associate_shapley_agent(N_M=N_M, approx_shapley_node=nested_shapley_node)

In [ ]:
print("For Case 1: Different agent contribution")
print("The Nested Shapley values of the agents are: \n")
for i, agent in enumerate(agents_list):
    print(f"Agent {agent}: {nested_shapley_agents[i].round(1)}")

### Case 2: Equal agent contribution

In [ ]:
# Calculate the characteristic function for case with all agents contributing the same
# Initialise v to save the characteristic functions of each subcoalition
v = np.zeros(len(A))
for row in range(len(A)):
    print(f"Run {row} of {len(A)}")
    current_agents_index = A[row]
    current_agents = [agent for i, agent in enumerate(agents_list) if current_agents_index[i]==1]

    n_houses = np.sum(current_agents == 1)

    # Create dictionary of data with function generate_data_dict()
    data = generate_data_dict_case_2(file_path_data, start_date_str, end_date_str, demand, current_agents)

    # Run the model
    instance = model_p2p(data)
    v[row] = instance.objective_function.expr()

# Activate to save the value functions in a csv file
#np.savetxt(r"v_NestedShapley.csv", v, delimiter=",", fmt="%f")

nodal_shapley = get_shapley_node(Ci, A, VS, v)
nested_shapley_node = get_approx_shapley_node(Ci, nodal_shapley)
nested_shapley_agents = associate_shapley_agent(N_M=N_M, approx_shapley_node=nested_shapley_node)

In [ ]:
print("For Case 2: Equal agent contribution")
print("The Nested Shapley values of the agents are: \n")
for i, agent in enumerate(agents_list):
    print(f"Agent {agent}: {nested_shapley_agents[i].round(1)}")

## Computations for tree 8-X

In [ ]:
# generate clusters according to tree
n_clusters = 8
sil_score_dict, sample_silhouette_values_dict, centers_dict = SilhouetteScoreRange(array=demand_array, nClusters = n_clusters)
dataKMeans, dataProcessed = KMeansAlgorithm(array=demand_array, dataframe=demand_, nClusters=n_clusters)

# Get agents ordered and number of agents in each node
agents_list, agents = agents_preprocessing(dataKMeans) # agents_list contains the name of the agent and the element it corresponds
demand = demand[agents_list] # reorganise the demand to match the clustering tree

# Prepare ordered tree. This case manually created based on the results from the k-means algorithm
Ci_list = [
    [2,3,4,5,6,7,8,9],
    [10, 11, 12],
    [13, 14],
    [15, 16],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    []
]

Ci = get_Ci_matrix(Ci_list)

N_M_list = [
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 [1, 2, 3],
 [4, 5],
 [6, 7],
 [8],
 [9],
 [10],
 [11],
 [12],
 [1],
 [2],
 [3],
 [4],
 [5],
 [6],
 [7]]

# Construct the necessary matrices for computing appropriately the Nested Shapley. See basic_shapley.py for a simple example.
N_M_list = create_N_M_list(Ci_list, Ci)
N_M = get_NM_matrix(N_M_list)

VS = get_VS_matrix(Ci)

A = np.dot(VS,N_M)

### Case 1: Different agent contribution

In [ ]:
# Calculate the characteristic function
# Initialise v to save the characteristic functions of each subcoalition
v = np.zeros(len(A))
for row in range(len(A)):
    print(f"Run {row} of {len(A)}")
    current_agents_index = A[row]
    current_agents = [agent for i, agent in enumerate(agents_list) if current_agents_index[i]==1]

    n_houses = np.sum(current_agents == 1)

    # Create dictionary of data with function generate_data_dict_case_1()
    data = generate_data_dict_case_1(file_path_data, start_date_str, end_date_str, demand, current_agents, houses_pv, houses_bat)

    # Run the model
    instance = model_p2p(data)
    v[row] = instance.objective_function.expr()

# Activate to save the value functions in a csv file
#np.savetxt(r"v_NestedShapley.csv", v, delimiter=",", fmt="%f")

nodal_shapley = get_shapley_node(Ci, A, VS, v)
nested_shapley_node = get_approx_shapley_node(Ci, nodal_shapley)
nested_shapley_agents = associate_shapley_agent(N_M=N_M, approx_shapley_node=nested_shapley_node)

In [ ]:
print("For Case 1: Different agent contribution")
print("The Nested Shapley values of the agents are: \n")
for i, agent in enumerate(agents_list):
    print(f"Agent {agent}: {nested_shapley_agents[i].round(1)}")

### Case 2: Equal agent contribution

In [ ]:
# Calculate the characteristic function for case with all agents contributing the same
# Initialise v to save the characteristic functions of each subcoalition
v = np.zeros(len(A))
for row in range(len(A)):
    print(f"Run {row} of {len(A)}")
    current_agents_index = A[row]
    current_agents = [agent for i, agent in enumerate(agents_list) if current_agents_index[i]==1]

    n_houses = np.sum(current_agents == 1)

    # Create dictionary of data with function generate_data_dict()
    data = generate_data_dict_case_2(file_path_data, start_date_str, end_date_str, demand, current_agents)

    # Run the model
    instance = model_p2p(data)
    v[row] = instance.objective_function.expr()

# Activate to save the value functions in a csv file
#np.savetxt(r"v_NestedShapley.csv", v, delimiter=",", fmt="%f")

nodal_shapley = get_shapley_node(Ci, A, VS, v)
nested_shapley_node = get_approx_shapley_node(Ci, nodal_shapley)
nested_shapley_agents = associate_shapley_agent(N_M=N_M, approx_shapley_node=nested_shapley_node)

In [ ]:
print("For Case 2: Equal agent contribution")
print("The Nested Shapley values of the agents are: \n")
for i, agent in enumerate(agents_list):
    print(f"Agent {agent}: {nested_shapley_agents[i].round(1)}")